In [1]:
!pip install pandarallel
import pandas as pd
pd.set_option("max_colwidth", 600)
from bs4 import BeautifulSoup
import requests

from pandarallel import pandarallel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.2-py3-none-any.whl size=16411 sha256=745ae8dff89fb901de16f23438ca5d0602385243f2cca6b0b49af34e4eac476c
  Stored in directory: /root/.cache/pip/wheels/01/8d/99/4ce6a1e914ed6ec383bc099dfbe0112dfde0bc39957ca93ec1
Successfully built pandarallel


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
def extract_urls_topic(url):
    try:
      coverpage = requests.get(url).content
      soup = BeautifulSoup(coverpage, 'html.parser')
     
      rows = []
      coverpage=soup.find_all('loc')
      for link in coverpage:
        if link.get_text().find('area_metropolitana')>=0 or link.get_text().find('provincia')>=0:
          rows.append(link.get_text()) 
        else :
          if link.get_text().find('www.granadahoy.com/granada/')>=0:
            rows.append(link.get_text()) 
          else :
            if link.get_text().find('causa-criminal')>=0:
              rows.append(link.get_text())
    except:
      rows = 'error'
      
    return rows

In [7]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)

try:
  coverpage = requests.get('https://www.granadahoy.com/sitemap_index.xml').content
  soup = BeautifulSoup(coverpage, 'html.parser')
  
  rows =[]
  coverpage=soup.find_all('loc')
  for link in coverpage:
    rows.append(link.get_text()) 
except:
  rows = 'error'
df = pd.DataFrame(range(0,182),columns = ['page'])
df['page']=rows
print(df)

df['rows']=df['page'].parallel_apply(extract_urls_topic)
df = df[df.rows.str.len()!=0].reset_index(drop=True)
df = df.explode('rows', ignore_index=True)
df=df.drop_duplicates(subset='rows')
df = df.dropna()
print(df)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Granadahoy/url.csv',index=False)

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
                                                        page
0             https://www.granadahoy.com/sitemap_authors.xml
1    https://www.granadahoy.com/sitemap_contents_2007_11.xml
2    https://www.granadahoy.com/sitemap_contents_2007_12.xml
3    https://www.granadahoy.com/sitemap_contents_2008_01.xml
4    https://www.granadahoy.com/sitemap_contents_2008_02.xml
..                                                       ...
177  https://www.granadahoy.com/sitemap_contents_2022_07.xml
178  https://www.granadahoy.com/sitemap_contents_2022_08.xml
179        https://www.granadahoy.com/sitemap_microsites.xml
180          https://www.granadahoy.com/sitemap_sections.xml
181              https://www.granadahoy.com/sitemap_tags.xml

[182 rows x 1 columns]


                                                           page  \
0       https://www.granadahoy.com/sitemap_contents_2007_11.xml   
1       https://www.granadahoy.com/sitemap_contents_2007_11.xml   
2       https://www.granadahoy.com/sitemap_contents_2007_11.xml   
3       https://www.granadahoy.com/sitemap_contents_2007_11.xml   
4       https://www.granadahoy.com/sitemap_contents_2007_11.xml   
...                                                         ...   
160334              https://www.granadahoy.com/sitemap_tags.xml   
160335              https://www.granadahoy.com/sitemap_tags.xml   
160336              https://www.granadahoy.com/sitemap_tags.xml   
160337              https://www.granadahoy.com/sitemap_tags.xml   
160338              https://www.granadahoy.com/sitemap_tags.xml   

                                                                                                                rows  
0       http://www.granadahoy.com/granada/protestan-identificacion-Piden-alt